In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from DataFields import DataFields
from DataFields import DateReportedFields
from UtilityFunctions import get_columns_from_chunk
from UtilityFunctions import convert_date_to_binary
from UtilityFunctions import count_na_in_dataframe

In [ ]:
diagnosed_df = pd.read_csv("diagnosed.csv")

# convert field which use write date a disease is reported and mark it as true or false
# to determine if a person was diagnosed with the diseas or not rather than when.
diagnosed_df = convert_date_to_binary(diagnosed_df, DateReportedFields)

diagnosed_df = diagnosed_df.drop(columns=["Unnamed: 0.1", "Unnamed: 0", "id"])
count_na_in_dataframe(diagnosed_df)

In [ ]:
cols = ["Oestradiol", "Rheumatoid factor", "Recent trouble concentrating on things",
        "Bipolar or Major Depression", "Ever had prolonged feelings of sadness or depression"]

diagnosed_df = diagnosed_df.drop(columns=cols)

In [ ]:
diagnosed_df.to_csv("diagnosed_processed.csv")